# Data Cleaning, Integration & Feature Selection

In [40]:
# Import Libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore")

## Reviews Data

In [2]:
# IMPORT DATASET
reviews = pd.read_csv('reviews-2.csv')
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,49091,8243238,2013-10-21,8557223,Jared,Fran was absolutely gracious and welcoming. Ma...
1,50646,11909864,2014-04-18,1356099,James,A comfortable room in a smart condo developmen...
2,50646,13823948,2014-06-05,15222393,Welli,Stayed over at Sujatha's house for 3 good nigh...
3,50646,15117222,2014-07-02,5543172,Cyril,It's been a lovely stay at Sujatha's. The room...
4,50646,15426462,2014-07-08,817532,Jake,"We had a great experience. A nice place, an am..."


In [3]:
# REMOVE NaN VALUES AND EMPTY STRINGS:
reviews.isnull().sum()    
reviews.dropna(inplace=True)
reviews.isnull().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

In [4]:
# INITIALIZE SENTIMENT ANALYZER
sia = SentimentIntensityAnalyzer()

In [5]:
# CALCULATE POLARITY SCORE
reviews['compound'] = [sia.polarity_scores(x)['compound'] for x in reviews['comments']]
reviews['neg'] = [sia.polarity_scores(x)['neg'] for x in reviews['comments']]
reviews['neu'] = [sia.polarity_scores(x)['neu'] for x in reviews['comments']]
reviews['pos'] = [sia.polarity_scores(x)['pos'] for x in reviews['comments']]

In [6]:
reviews.describe() 

,listing_id,id,reviewer_id,compound,neg,neu,pos
count,6.144500e+04,6.144500e+04,6.144500e+04,61445.000000,61445.000000,61445.00000,61445.000000
mean,1.700946e+07,3.548177e+08,1.059403e+08,0.635832,0.012094,0.69706,0.290828
std,1.107024e+07,1.916486e+08,9.124109e+07,0.396951,0.041405,0.21565,0.219641
min,4.909100e+04,2.514670e+05,2.770000e+02,-0.998300,0.000000,0.00000,0.000000
25%,7.808030e+06,1.898209e+08,3.034613e+07,0.440400,0.000000,0.58000,0.139000
50%,1.616440e+07,3.551213e+08,7.829610e+07,0.817200,0.000000,0.71800,0.269000
75%,2.450665e+07,5.324980e+08,1.611622e+08,0.936600,0.000000,0.83300,0.412000
max,4.581814e+07,7.040802e+08,3.727533e+08,0.999100,1.000000,1.00000,1.000000


In [61]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,compound,neg,neu,pos,avgcomp
0,49091,8243238,2013-10-21,8557223,Jared,Fran was absolutely gracious and welcoming. Ma...,0.9615,0.0,0.450,0.550,0.961500
1,50646,11909864,2014-04-18,1356099,James,A comfortable room in a smart condo developmen...,0.9484,0.0,0.672,0.328,0.825117
2,50646,13823948,2014-06-05,15222393,Welli,Stayed over at Sujatha's house for 3 good nigh...,0.9881,0.0,0.616,0.384,0.825117
3,50646,15117222,2014-07-02,5543172,Cyril,It's been a lovely stay at Sujatha's. The room...,0.9627,0.0,0.645,0.355,0.825117
4,50646,15426462,2014-07-08,817532,Jake,"We had a great experience. A nice place, an am...",0.9274,0.0,0.489,0.511,0.825117


In [7]:
# CALCULATE AVERAGE SCORES FOR A SINGLE LISTING (USING LISTING ID)
reviews['avgcomp'] = reviews.groupby('listing_id')['compound'].transform('mean')

In [8]:
# DROP DUPLICATES OF LISTING ID
cleanreviews = reviews.drop_duplicates(subset='listing_id', keep='first', inplace=False)

In [9]:
# DROP UNNECESSARY COLUMNS
cleanreviews = cleanreviews.drop(['id',
                                  'date', 
                                  'reviewer_id', 
                                  'reviewer_name', 
                                  'comments', 
                                  'compound',
                                  'neg',
                                  'neu',
                                  'pos'], axis = 1) 

In [10]:
cleanreviews.head()

,listing_id,avgcomp
0,49091,0.961500
1,50646,0.825117
19,56334,0.899190
39,71609,0.577285
59,71896,0.747200


In [11]:
# EXPORT CLEANED REVIEW DATA TO CSV FILE
cleanreviews.to_csv('cleanreviews.csv', header=True)

## Listings Data 
### a) Listing Data without Feature Selection

In [12]:
# IMPORT LISTING DATASET
listings = pd.read_csv('listings-2.csv')
listings.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,49091,https://www.airbnb.com/rooms/49091,20201026165207,2020-10-26,COZICOMFORT LONG TERM STAY ROOM 2,<b>The space</b><br />This is Room No. 2.(avai...,NaN,https://a0.muscache.com/pictures/24485541/67b0...,266763,https://www.airbnb.com/users/show/266763,...,10.0,8.0,8.0,NaN,f,2,0,2,0,0.01
1,50646,https://www.airbnb.com/rooms/50646,20201026165207,2020-10-26,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,https://a0.muscache.com/pictures/36342984/670d...,227796,https://www.airbnb.com/users/show/227796,...,10.0,9.0,9.0,NaN,f,1,0,1,0,0.23
2,56334,https://www.airbnb.com/rooms/56334,20201026165207,2020-10-26,COZICOMFORT,<b>The space</b><br />This is Room no. 1. USD7...,NaN,https://a0.muscache.com/pictures/20783922/1966...,266763,https://www.airbnb.com/users/show/266763,...,10.0,8.0,9.0,NaN,f,2,0,2,0,0.18
3,71609,https://www.airbnb.com/rooms/71609,20201026165207,2020-10-26,Ensuite Room (Room 1 & 2) near EXPO,For 3 rooms.Book room 1&2 and room 4<br /><br ...,NaN,https://a0.muscache.com/pictures/24453191/3580...,367042,https://www.airbnb.com/users/show/367042,...,10.0,9.0,9.0,NaN,t,8,0,8,0,0.19
4,71896,https://www.airbnb.com/rooms/71896,20201026165207,2020-10-26,B&B Room 1 near Airport & EXPO,<b>The space</b><br />Vocational Stay Deluxe B...,NaN,https://a0.muscache.com/pictures/2440674/ac4f4...,367042,https://www.airbnb.com/users/show/367042,...,9.0,8.0,8.0,NaN,t,8,0,8,0,0.21


In [13]:
# DROP UNNECCESSARY COLUMNS
cleanlistings = listings.drop(['listing_url', 'scrape_id', 'last_scraped', 'name', 'description', 
                               'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 
                               'host_since', 'host_name', 'host_location', 'host_about', 
                               'host_response_time', 'host_acceptance_rate', 'host_thumbnail_url', 
                               'host_picture_url', 'host_neighbourhood', 'host_listings_count', 
                               'host_verifications',  
                               'neighbourhood',    'bathrooms','amenities',
                               'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
                               'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
                               'calendar_updated', 'availability_30', 'availability_60', 
                               'availability_90', 'availability_365', 'calendar_last_scraped',  
                               'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 
                               'last_review', 'review_scores_rating', 'review_scores_accuracy',
                               'review_scores_cleanliness', 'review_scores_checkin',
                               'review_scores_communication', 'review_scores_location', 'review_scores_value',
                               'license', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
                               'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 
                               'reviews_per_month'], axis = 1) 

In [14]:
# CHECK MISSING VALUES
cleanlistings.isnull().sum()

id                                0
host_response_rate              599
host_is_superhost                 2
host_total_listings_count         2
host_has_profile_pic              2
host_identity_verified            2
neighbourhood_cleansed            0
neighbourhood_group_cleansed      0
latitude                          0
longitude                         0
property_type                     0
room_type                         0
accommodates                      0
bathrooms_text                   14
bedrooms                        386
beds                             55
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
number_of_reviews                 0
instant_bookable                  0
dtype: int64

In [15]:
# REPLACE MISSING VALUES FOR 'host_is_superhost'
cleanlistings['host_is_superhost'] = cleanlistings['host_is_superhost'].replace(np.nan, 'f')

In [16]:
# CONVERT T/F VALUES TO BOOLEAN
cleanlistings.replace({'f': 0, 't': 1}, inplace=True)

In [17]:
# CLEAN PRICE DATA
cleanlistings['price'] = cleanlistings['price'].str.replace(',', '')
cleanlistings['price'] = cleanlistings['price'].str.replace('$', '')
cleanlistings['price'] = cleanlistings['price'].astype(float)

In [18]:
# CLEAN RESPONSE RATE DATA
cleanlistings['host_response_rate'] = cleanlistings['host_response_rate'].str.replace('%', '')
cleanlistings['host_response_rate'] = cleanlistings['host_response_rate'].replace(np.nan, '0')
cleanlistings['host_response_rate'] = cleanlistings['host_response_rate'].astype(int)

In [19]:
# CLEAN BATHROOM_TEXT DATA
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace(' baths', '')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace(' bath', '')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace(' private', '')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace(' shared', '')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace('Shared half-bath', '0.5')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace('Private half-bath', '0.5')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].str.replace('Half-bath', '0.5')
cleanlistings['bathrooms_text'] = cleanlistings['bathrooms_text'].astype(float)

In [20]:
# CHECK FOR NULL VALUES
cleanlistings.isnull().sum()

id                                0
host_response_rate                0
host_is_superhost                 0
host_total_listings_count         2
host_has_profile_pic              2
host_identity_verified            2
neighbourhood_cleansed            0
neighbourhood_group_cleansed      0
latitude                          0
longitude                         0
property_type                     0
room_type                         0
accommodates                      0
bathrooms_text                   14
bedrooms                        386
beds                             55
price                             0
minimum_nights                    0
maximum_nights                    0
has_availability                  0
number_of_reviews                 0
instant_bookable                  0
dtype: int64

In [21]:
cleanlistings.head()

,id,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,...,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable
0,49091,100,0,2.0,1.0,1.0,Woodlands,North Region,1.44255,103.79580,...,1,1.0,1.0,1.0,82.0,180,360,1,1,0
1,50646,0,0,1.0,1.0,1.0,Bukit Timah,Central Region,1.33235,103.78521,...,2,1.0,1.0,1.0,80.0,90,730,1,18,0
2,56334,100,0,2.0,1.0,1.0,Woodlands,North Region,1.44246,103.79667,...,1,1.0,1.0,1.0,68.0,6,14,1,20,0
3,71609,100,0,8.0,1.0,1.0,Tampines,East Region,1.34541,103.95712,...,6,1.0,2.0,3.0,179.0,90,1125,1,20,1
4,71896,100,0,8.0,1.0,1.0,Tampines,East Region,1.34567,103.95963,...,3,0.5,1.0,1.0,95.0,90,1125,1,24,1


In [22]:
# DROPPING ROWS WITH NULL VALUES
cleanlistings = cleanlistings.dropna()

In [23]:
# CHECK FOR NULL VALUES
cleanlistings.isnull().sum()

id                              0
host_response_rate              0
host_is_superhost               0
host_total_listings_count       0
host_has_profile_pic            0
host_identity_verified          0
neighbourhood_cleansed          0
neighbourhood_group_cleansed    0
latitude                        0
longitude                       0
property_type                   0
room_type                       0
accommodates                    0
bathrooms_text                  0
bedrooms                        0
beds                            0
price                           0
minimum_nights                  0
maximum_nights                  0
has_availability                0
number_of_reviews               0
instant_bookable                0
dtype: int64

In [24]:
cleanlistings.head()

,id,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,...,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable
0,49091,100,0,2.0,1.0,1.0,Woodlands,North Region,1.44255,103.79580,...,1,1.0,1.0,1.0,82.0,180,360,1,1,0
1,50646,0,0,1.0,1.0,1.0,Bukit Timah,Central Region,1.33235,103.78521,...,2,1.0,1.0,1.0,80.0,90,730,1,18,0
2,56334,100,0,2.0,1.0,1.0,Woodlands,North Region,1.44246,103.79667,...,1,1.0,1.0,1.0,68.0,6,14,1,20,0
3,71609,100,0,8.0,1.0,1.0,Tampines,East Region,1.34541,103.95712,...,6,1.0,2.0,3.0,179.0,90,1125,1,20,1
4,71896,100,0,8.0,1.0,1.0,Tampines,East Region,1.34567,103.95963,...,3,0.5,1.0,1.0,95.0,90,1125,1,24,1


In [25]:
# TRANSFORM CATEGORICAL TO NUMERICAL VALUE USING LABEL ENCODER
le = LabelEncoder()
cleanlistings["ptype_code"] = le.fit_transform(cleanlistings["property_type"])
cleanlistings["rtype_code"] = le.fit_transform(cleanlistings["room_type"])
cleanlistings["ngtype_code"] = le.fit_transform(cleanlistings["neighbourhood_group_cleansed"])
cleanlistings["nbtype_code"] = le.fit_transform(cleanlistings["neighbourhood_cleansed"])

In [31]:
# CARRY OUT STANDARDIZATION ON NUMERICAL COLUMNS 
scaler = StandardScaler() 
numerical_cols = ['ptype_code','rtype_code','ngtype_code', 'nbtype_code']
cleanlistings[numerical_cols] = scaler.fit_transform(cleanlistings[numerical_cols])
cleanlistings['price'] = np.log(cleanlistings['price'])

In [32]:
# REMOVE IRRELEVANT COLUMNS
cleanlistings = cleanlistings.drop(['neighbourhood_cleansed', 'property_type', 'room_type','neighbourhood_group_cleansed'], axis = 1) 

In [33]:
cleanlistings.head()

,id,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,...,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code
0,49091,100,0,2.0,1.0,1.0,1.44255,103.79580,1,1.0,...,4.406719,180,360,1,1,0,-0.016218,0.849882,1.326759,2.181841
1,50646,0,0,1.0,1.0,1.0,1.33235,103.78521,2,1.0,...,4.382027,90,730,1,18,0,-0.016218,0.849882,-0.434121,-1.296388
2,56334,100,0,2.0,1.0,1.0,1.44246,103.79667,1,1.0,...,4.219508,6,14,1,20,0,-0.016218,0.849882,1.326759,2.181841
3,71609,100,0,8.0,1.0,1.0,1.34541,103.95712,6,1.0,...,5.187386,90,1125,1,20,1,1.242553,0.849882,0.446319,1.684951
4,71896,100,0,8.0,1.0,1.0,1.34567,103.95963,3,0.5,...,4.553877,90,1125,1,24,1,0.758410,0.849882,0.446319,1.684951


In [62]:
cleanlistings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4042 entries, 0 to 4491
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         4042 non-null   int64  
 1   host_response_rate         4042 non-null   int64  
 2   host_is_superhost          4042 non-null   int64  
 3   host_total_listings_count  4042 non-null   float64
 4   host_has_profile_pic       4042 non-null   float64
 5   host_identity_verified     4042 non-null   float64
 6   latitude                   4042 non-null   float64
 7   longitude                  4042 non-null   float64
 8   accommodates               4042 non-null   int64  
 9   bathrooms_text             4042 non-null   float64
 10  bedrooms                   4042 non-null   float64
 11  beds                       4042 non-null   float64
 12  price                      4042 non-null   float64
 13  minimum_nights             4042 non-null   int64

In [34]:
# EXPORT TO CSV
cleanlistings.to_csv('a.csv', header=True)

### b) Listing Data with Feature Selection

In [35]:
# BACKWARD ELIMINATION FUNCTION
def backward_elimination(data, target,significance_level = 0.05):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

In [36]:
# CREATE NEW COPY OF LISTINGS DATA
cleanlistings2 = cleanlistings.copy()
cleanlistings2.head()

,id,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,...,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code
0,49091,100,0,2.0,1.0,1.0,1.44255,103.79580,1,1.0,...,4.406719,180,360,1,1,0,-0.016218,0.849882,1.326759,2.181841
1,50646,0,0,1.0,1.0,1.0,1.33235,103.78521,2,1.0,...,4.382027,90,730,1,18,0,-0.016218,0.849882,-0.434121,-1.296388
2,56334,100,0,2.0,1.0,1.0,1.44246,103.79667,1,1.0,...,4.219508,6,14,1,20,0,-0.016218,0.849882,1.326759,2.181841
3,71609,100,0,8.0,1.0,1.0,1.34541,103.95712,6,1.0,...,5.187386,90,1125,1,20,1,1.242553,0.849882,0.446319,1.684951
4,71896,100,0,8.0,1.0,1.0,1.34567,103.95963,3,0.5,...,4.553877,90,1125,1,24,1,0.758410,0.849882,0.446319,1.684951


In [37]:
# DROP UNNECCESSARY COLUMNS
cleanlistings2 = cleanlistings2.drop(columns='id')
cleanlistings2.head()

,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,bedrooms,...,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code
0,100,0,2.0,1.0,1.0,1.44255,103.79580,1,1.0,1.0,...,4.406719,180,360,1,1,0,-0.016218,0.849882,1.326759,2.181841
1,0,0,1.0,1.0,1.0,1.33235,103.78521,2,1.0,1.0,...,4.382027,90,730,1,18,0,-0.016218,0.849882,-0.434121,-1.296388
2,100,0,2.0,1.0,1.0,1.44246,103.79667,1,1.0,1.0,...,4.219508,6,14,1,20,0,-0.016218,0.849882,1.326759,2.181841
3,100,0,8.0,1.0,1.0,1.34541,103.95712,6,1.0,2.0,...,5.187386,90,1125,1,20,1,1.242553,0.849882,0.446319,1.684951
4,100,0,8.0,1.0,1.0,1.34567,103.95963,3,0.5,1.0,...,4.553877,90,1125,1,24,1,0.758410,0.849882,0.446319,1.684951


In [38]:
# SEPARATE OTHER ATTRIBUTE FROM PREDICTING ATTRIBUTE
X = cleanlistings2.drop('price',axis=1)
y = cleanlistings2['price']

In [41]:
backward_elimination(X,y)

['host_response_rate',
 'host_is_superhost',
 'latitude',
 'accommodates',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'minimum_nights',
 'has_availability',
 'number_of_reviews',
 'instant_bookable',
 'ptype_code',
 'rtype_code',
 'ngtype_code',
 'nbtype_code']

In [42]:
cleanlistings2 = cleanlistings2[['price', 
                                 'host_response_rate',
                                 'host_is_superhost',
                                 'latitude',
                                 'accommodates',
                                 'bathrooms_text',
                                 'bedrooms',
                                 'beds',
                                 'minimum_nights',
                                 'has_availability',
                                 'number_of_reviews',
                                 'instant_bookable',
                                 'ptype_code',
                                 'rtype_code',
                                 'ngtype_code',
                                 'nbtype_code'
                               ]]

In [43]:
cleanlistings2.head()

,price,host_response_rate,host_is_superhost,latitude,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code
0,4.406719,100,0,1.44255,1,1.0,1.0,1.0,180,1,1,0,-0.016218,0.849882,1.326759,2.181841
1,4.382027,0,0,1.33235,2,1.0,1.0,1.0,90,1,18,0,-0.016218,0.849882,-0.434121,-1.296388
2,4.219508,100,0,1.44246,1,1.0,1.0,1.0,6,1,20,0,-0.016218,0.849882,1.326759,2.181841
3,5.187386,100,0,1.34541,6,1.0,2.0,3.0,90,1,20,1,1.242553,0.849882,0.446319,1.684951
4,4.553877,100,0,1.34567,3,0.5,1.0,1.0,90,1,24,1,0.758410,0.849882,0.446319,1.684951


In [63]:
cleanlistings2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4042 entries, 0 to 4491
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               4042 non-null   float64
 1   host_response_rate  4042 non-null   int64  
 2   host_is_superhost   4042 non-null   int64  
 3   latitude            4042 non-null   float64
 4   accommodates        4042 non-null   int64  
 5   bathrooms_text      4042 non-null   float64
 6   bedrooms            4042 non-null   float64
 7   beds                4042 non-null   float64
 8   minimum_nights      4042 non-null   int64  
 9   has_availability    4042 non-null   int64  
 10  number_of_reviews   4042 non-null   int64  
 11  instant_bookable    4042 non-null   int64  
 12  ptype_code          4042 non-null   float64
 13  rtype_code          4042 non-null   float64
 14  ngtype_code         4042 non-null   float64
 15  nbtype_code         4042 non-null   float64
dtypes: flo

In [60]:
# EXPORT LISTING DATA WITH FEATURE SELECTED
cleanlistings2.to_csv('b.csv', header=True)

### c) Listing & Review Data without Feature Selection

In [45]:
# CREATE NEW COPY OF LISTINGS DATA
cleanlistings3 = cleanlistings.copy()
cleanlistings3.head()

,id,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,...,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code
0,49091,100,0,2.0,1.0,1.0,1.44255,103.79580,1,1.0,...,4.406719,180,360,1,1,0,-0.016218,0.849882,1.326759,2.181841
1,50646,0,0,1.0,1.0,1.0,1.33235,103.78521,2,1.0,...,4.382027,90,730,1,18,0,-0.016218,0.849882,-0.434121,-1.296388
2,56334,100,0,2.0,1.0,1.0,1.44246,103.79667,1,1.0,...,4.219508,6,14,1,20,0,-0.016218,0.849882,1.326759,2.181841
3,71609,100,0,8.0,1.0,1.0,1.34541,103.95712,6,1.0,...,5.187386,90,1125,1,20,1,1.242553,0.849882,0.446319,1.684951
4,71896,100,0,8.0,1.0,1.0,1.34567,103.95963,3,0.5,...,4.553877,90,1125,1,24,1,0.758410,0.849882,0.446319,1.684951


In [46]:
# INTEGRATE DATA USING ID & LISTING ID
integdata = pd.merge(cleanlistings3, cleanreviews, left_on='id', right_on='listing_id', how='right')

In [47]:
integdata.head()

,id,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,...,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code,listing_id,avgcomp
0,49091.0,100.0,0.0,2.0,1.0,1.0,1.44255,103.79580,1.0,1.0,...,360.0,1.0,1.0,0.0,-0.016218,0.849882,1.326759,2.181841,49091,0.961500
1,50646.0,0.0,0.0,1.0,1.0,1.0,1.33235,103.78521,2.0,1.0,...,730.0,1.0,18.0,0.0,-0.016218,0.849882,-0.434121,-1.296388,50646,0.825117
2,56334.0,100.0,0.0,2.0,1.0,1.0,1.44246,103.79667,1.0,1.0,...,14.0,1.0,20.0,0.0,-0.016218,0.849882,1.326759,2.181841,56334,0.899190
3,71609.0,100.0,0.0,8.0,1.0,1.0,1.34541,103.95712,6.0,1.0,...,1125.0,1.0,20.0,1.0,1.242553,0.849882,0.446319,1.684951,71609,0.577285
4,71896.0,100.0,0.0,8.0,1.0,1.0,1.34567,103.95963,3.0,0.5,...,1125.0,1.0,24.0,1.0,0.758410,0.849882,0.446319,1.684951,71896,0.747200


In [48]:
integdata.isnull().sum()

id                           273
host_response_rate           273
host_is_superhost            273
host_total_listings_count    273
host_has_profile_pic         273
host_identity_verified       273
latitude                     273
longitude                    273
accommodates                 273
bathrooms_text               273
bedrooms                     273
beds                         273
price                        273
minimum_nights               273
maximum_nights               273
has_availability             273
number_of_reviews            273
instant_bookable             273
ptype_code                   273
rtype_code                   273
ngtype_code                  273
nbtype_code                  273
listing_id                     0
avgcomp                        0
dtype: int64

In [49]:
# DROP DATA THAT DOESN'T HAVE REVIEW DATA
integdata = integdata.dropna()

In [50]:
# DROP ID AND LISTING ID
integdata = integdata.drop(columns='id')
integdata = integdata.drop(columns='listing_id')

In [51]:
integdata.head()

,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,bedrooms,...,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code,avgcomp
0,100.0,0.0,2.0,1.0,1.0,1.44255,103.79580,1.0,1.0,1.0,...,180.0,360.0,1.0,1.0,0.0,-0.016218,0.849882,1.326759,2.181841,0.961500
1,0.0,0.0,1.0,1.0,1.0,1.33235,103.78521,2.0,1.0,1.0,...,90.0,730.0,1.0,18.0,0.0,-0.016218,0.849882,-0.434121,-1.296388,0.825117
2,100.0,0.0,2.0,1.0,1.0,1.44246,103.79667,1.0,1.0,1.0,...,6.0,14.0,1.0,20.0,0.0,-0.016218,0.849882,1.326759,2.181841,0.899190
3,100.0,0.0,8.0,1.0,1.0,1.34541,103.95712,6.0,1.0,2.0,...,90.0,1125.0,1.0,20.0,1.0,1.242553,0.849882,0.446319,1.684951,0.577285
4,100.0,0.0,8.0,1.0,1.0,1.34567,103.95963,3.0,0.5,1.0,...,90.0,1125.0,1.0,24.0,1.0,0.758410,0.849882,0.446319,1.684951,0.747200


In [64]:
integdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2419 entries, 0 to 2691
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   host_response_rate         2419 non-null   float64
 1   host_is_superhost          2419 non-null   float64
 2   host_total_listings_count  2419 non-null   float64
 3   host_has_profile_pic       2419 non-null   float64
 4   host_identity_verified     2419 non-null   float64
 5   latitude                   2419 non-null   float64
 6   longitude                  2419 non-null   float64
 7   accommodates               2419 non-null   float64
 8   bathrooms_text             2419 non-null   float64
 9   bedrooms                   2419 non-null   float64
 10  beds                       2419 non-null   float64
 11  price                      2419 non-null   float64
 12  minimum_nights             2419 non-null   float64
 13  maximum_nights             2419 non-null   float

In [52]:
# EXPORT DATA TO CSV
integdata.to_csv('c.csv', header=True)

### d) Listing & Review Data with Feature Selection

In [53]:
# CREATE NEW COPY OF INTEGRATED DATA
integdata2 = integdata.copy()
integdata2.head()

,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms_text,bedrooms,...,minimum_nights,maximum_nights,has_availability,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code,avgcomp
0,100.0,0.0,2.0,1.0,1.0,1.44255,103.79580,1.0,1.0,1.0,...,180.0,360.0,1.0,1.0,0.0,-0.016218,0.849882,1.326759,2.181841,0.961500
1,0.0,0.0,1.0,1.0,1.0,1.33235,103.78521,2.0,1.0,1.0,...,90.0,730.0,1.0,18.0,0.0,-0.016218,0.849882,-0.434121,-1.296388,0.825117
2,100.0,0.0,2.0,1.0,1.0,1.44246,103.79667,1.0,1.0,1.0,...,6.0,14.0,1.0,20.0,0.0,-0.016218,0.849882,1.326759,2.181841,0.899190
3,100.0,0.0,8.0,1.0,1.0,1.34541,103.95712,6.0,1.0,2.0,...,90.0,1125.0,1.0,20.0,1.0,1.242553,0.849882,0.446319,1.684951,0.577285
4,100.0,0.0,8.0,1.0,1.0,1.34567,103.95963,3.0,0.5,1.0,...,90.0,1125.0,1.0,24.0,1.0,0.758410,0.849882,0.446319,1.684951,0.747200


In [54]:
# SEPARATE OTHER ATTRIBUTE FROM PREDICTING ATTRIBUTE
X = integdata2.drop('price',axis=1)
y = integdata2['price']

In [55]:
backward_elimination(X,y)

['host_response_rate',
 'host_is_superhost',
 'host_total_listings_count',
 'latitude',
 'accommodates',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'minimum_nights',
 'number_of_reviews',
 'instant_bookable',
 'ptype_code',
 'rtype_code',
 'ngtype_code',
 'nbtype_code']

In [57]:
integdata2 = integdata2[['price',
                     'host_response_rate',
                     'host_is_superhost',
                     'host_total_listings_count',
                     'latitude',
                     'accommodates',
                     'bathrooms_text',
                     'bedrooms',
                     'beds',
                     'minimum_nights',
                     'number_of_reviews',
                     'instant_bookable',
                     'ptype_code',
                     'rtype_code',
                     'ngtype_code',
                     'nbtype_code',
                     'avgcomp']]

In [58]:
# CHECK LISTING COLUMNS
integdata2.head()

,price,host_response_rate,host_is_superhost,host_total_listings_count,latitude,accommodates,bathrooms_text,bedrooms,beds,minimum_nights,number_of_reviews,instant_bookable,ptype_code,rtype_code,ngtype_code,nbtype_code,avgcomp
0,4.406719,100.0,0.0,2.0,1.44255,1.0,1.0,1.0,1.0,180.0,1.0,0.0,-0.016218,0.849882,1.326759,2.181841,0.961500
1,4.382027,0.0,0.0,1.0,1.33235,2.0,1.0,1.0,1.0,90.0,18.0,0.0,-0.016218,0.849882,-0.434121,-1.296388,0.825117
2,4.219508,100.0,0.0,2.0,1.44246,1.0,1.0,1.0,1.0,6.0,20.0,0.0,-0.016218,0.849882,1.326759,2.181841,0.899190
3,5.187386,100.0,0.0,8.0,1.34541,6.0,1.0,2.0,3.0,90.0,20.0,1.0,1.242553,0.849882,0.446319,1.684951,0.577285
4,4.553877,100.0,0.0,8.0,1.34567,3.0,0.5,1.0,1.0,90.0,24.0,1.0,0.758410,0.849882,0.446319,1.684951,0.747200


In [66]:
integdata2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2419 entries, 0 to 2691
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   price                      2419 non-null   float64
 1   host_response_rate         2419 non-null   float64
 2   host_is_superhost          2419 non-null   float64
 3   host_total_listings_count  2419 non-null   float64
 4   latitude                   2419 non-null   float64
 5   accommodates               2419 non-null   float64
 6   bathrooms_text             2419 non-null   float64
 7   bedrooms                   2419 non-null   float64
 8   beds                       2419 non-null   float64
 9   minimum_nights             2419 non-null   float64
 10  number_of_reviews          2419 non-null   float64
 11  instant_bookable           2419 non-null   float64
 12  ptype_code                 2419 non-null   float64
 13  rtype_code                 2419 non-null   float

In [59]:
# EXPORT LISTING DATA WITH FEATURE SELECTED
integdata2.to_csv('d.csv', header=True)